### 3.1 keras 函数式api
#### 一. Sequential模型的局限
1. keras的Sequential模型, 只能处理一个输入, 产生一个输出. 其网络结构的组成是层之间的线性堆叠
2. 多输入的情况  
 假设一个场景: 预测二手物品价格. 其输入可能来自3个方面:  
  1. 用户提供的物品feature和对应价格的数字数据 - one hot后监督学习
  2. 对物品的文本描述和其价格 - RNN或1Dconvnet最后回归    
  3. 一张物品的图片和其价格 - 2dconvnet  
  
 此场景下, Sequential下需要独立的3个网络处理不同输入, 但是三个网络会从是哪个输入中学习到冗余的信息  
 更精确的做法是, 同时接受这三方面的信息并组合使用, 得到更精确的结果.  
 <img src='img/3inputsmodel.png' width='40%' height='40%'>
3. 多输出的情况  
 假设一个场景: 给定一个文章, 要识别文章的主题情感(喜剧/悲剧), 或者估计出文章大概创作的日期.  
 如果我们能有一个网络, 同时识别主题情感和创作日期, 则会得到更准确的结果. 因为文章的创作日期可以作为文章的representation, 来提升主题情感分类的准确度
 <img src='img/multioutputsfun.png' width='30%' height='30%'>
4. 有向无环图网络模型  
  1. 现代的神经网络很大部分不是层之间的线性堆叠, 而是每层作为定点形成有向无环图(DAG)
  2. 某些情况下, layer(n)的产生结果可能会作为输入再给到layer(n+t), 形成冗余连接. 这也是Sequential模型做不到的  
  <img src='img/dagmodel.png'>

#### 二. 函数式api 
1. keras每个layers中的对象, 都是一个函数. 这个函数接收一个input_tensor, 返回一个output_tensor  
 ```python
f = Dense(32,activation='relu')
f(input_tensor)
```


#### 三. 多输入情况
1. 当layer有多个输入张量时, 可以组合这些张量, 通过`layers.add`或`layers.concatenate`
2. 如下例子, 展示了有2个输入的知识问答系统.输出1个单词的回答  
 问答系统通常含有两个输入:   
   1. 自然语言表示的问题描述  
   2. 一个文本片断(比如一个新闻报道)
<img src='img/qumodel.png' width='55%' height='55%'>

In [6]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500
question_length = 20
reference_length = 100
word_features = 100

# reference text
text_input = Input(shape=(reference_length,),dtype='int32',name='txt')
embedded_text = layers.Embedding(text_vocabulary_size,word_features)(text_input)
lstm_text = layers.LSTM(16)(embedded_text)

# question description
question_input = Input(shape=(question_length,),dtype='int32',name='question')
embedded_question = layers.Embedding(question_vocabulary_size,word_features)(question_input)
lstm_question = layers.LSTM(16)(embedded_question)

# concatenate
concatenated_tensor = layers.concatenate([lstm_text,lstm_question],axis=-1)
answer = layers.Dense(answer_vocabulary_size,activation='softmax')(concatenated_tensor)

# Model(Input_tensor, Output_tensor)
model = Model([text_input,question_input],answer)
model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['acc'])

In [ ]:
# fitmodel
import numpy as np
samples = 1000

text = np.random.randint(1,text_vocabulary_size,size=(samples,reference_length))
question = np.random.randint(1,question_vocabulary_size,size=(samples,question_length))
answers = np.random.randint(0,1,size=(samples,answer_vocabulary_size))

model.fit([text,question],answers,epochs=10,batch_size=32)
# model.fit({'txt':text,'question':question},answers,epochs=10,batch_size=32)

#### 四. 多输出的情况
1. 多输出时, 每个输出要有不同的输出层
2. 每个输出要有不同的损失函数, 此外还应对每个损失函数赋予权重, 平衡每个输出对最终梯度下降的影响效果
3. 如下例子, 针对网络上提交的帖子, 预测这个贴子提交人的年龄, 性别, 收入阶层3个属性  
<img src='img/multioutputs.png' width='55%' height='55%'>

In [9]:
income_level_num = 10
post_inputs = Input(shape=(None,),dtype='int32',name='posts')
embedding_inputs = layers.Embedding(10000,200)(post_inputs)
x = layers.Conv1D(256,5,activation='relu')(embedding_inputs)
x = layers.MaxPooling1D(5)(x)
x = layers.GlobalMaxPool1D()(x)

# 3个输出
age_prediction = layers.Dense(1,name='age')(x)
gender_prediction = layers.Dense(1,name='gender',activation='sigmoid')(x)
income_level_prediction = layers.Dense(income_level_num,activation='softmax',name='income')(x)

model = Model(post_inputs,[age_prediction,gender_prediction,income_level_prediction])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
posts (InputLayer)              (None, None)         0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, None, 200)    2000000     posts[0][0]                      
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 256)    256256      embedding_3[0][0]                
__________________________________________________________________________________________________
max_pooling1d_1 (MaxPooling1D)  (None, None, 256)    0           conv1d_1[0][0]                   
__________________________________________________________________________________________________
global_max

In [10]:
# 定义损失函数
model.compile(optimizer='rmsprop',
              # layer_name : loss_function
              loss = {'age':'mae','gender':'binary_crossentropy','income':'categorical_crossentropy'}, 
              loss_weight = {'age':0.25,'gender':10,'income':1})

#### 五. layer构成的有向无环图
1. 神经网络的有向无环图结构有2个值得注意的模式  
  1. inception model  
  2. residual connection  
 下图展示了2中不同的有向无环图网络结构
2. INCEPTION MODULES  
  1. INCEPTION网络是CNN网络的重要结构, 它由一组独立的模块堆叠形成, 像是把输入分割成多个并行计算的patch, 再讲其结果链接起来.
  具体过程查看[Inception work](http://nbviewer.jupyter.org/github/lj72808up/DeepLearning/blob/master/blog/3-ResNet%20vs%20Inception%20Net.ipynb)  
  2. keras中, 堆叠输出张量为一个张量的方法为`layers.concatenate`
<img src='img/inception.png' height='68%' width='68%'>

3. RESIDUAL CONNECTIONS  
  1. 通常跨10层进行layer之间的冗余连接.冗余连接解决了2个问题  
    1. 大规模神经网络下的梯度消失  
    2. 大规模神经网络下的表现力瓶颈
  2. 通常使用layers.add(tensor1,tensor2)把至少10层之前的输出tensor加到当前输出tensor上
<img src='img/residualconnect.png'  height='45%' width='45%'>

#### 六 共享layer权重  
1. keras函数式api的一个重要应用是layer权重共享.它允许1个layer实例被多次使用  
2. 应用场景 : 对话系统中常常有一个问题相似度比对. 输入两个问句, 如果两句话相似输出1, 不想死输出0  
 如下代码给出了这样的网络的搭建

In [12]:
# 共用的layer实例, 对立忙个输入执行完全一样的参数和运算
lstm = layers.LSTM(32)
# 左分支的输入
left_input = Input(shape=(None,128))
left_output = lstm(left_input)
# 右分支的输入
right_input = Input(shape=(None,128))
right_output = lstm(right_input)

merged = layers.concatenate([left_output,right_output],axis = -1)
predictions = layers.Dense(1,activation='sigmoid')(merged)

model = Model([left_input,right_input],predictions)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, None, 128)    0                                            
__________________________________________________________________________________________________
lstm_4 (LSTM)                   (None, 32)           20608       input_3[0][0]                    
                                                                 input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 64)           0           lstm_4[0][0]                     
          